In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fer2013


### Resnet34

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import zipfile
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report
from tqdm import tqdm

# -- Dataset Setup --
BASE_DIR = '/kaggle/input/fer2013'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

entries = os.listdir(BASE_DIR)
print("Entries in BASE_DIR:", entries)

# Folder-based structure
if 'train' in entries and 'test' in entries:
    print("Using folder-based dataset...")
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((48,48)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_folder = os.path.join(BASE_DIR, 'train')
    test_folder  = os.path.join(BASE_DIR, 'test')
    full_train = ImageFolder(train_folder, transform=transform)
    train_size = int(0.9 * len(full_train))
    val_size   = len(full_train) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_train, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )
    test_dataset = ImageFolder(test_folder, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_dataset,   batch_size=128, shuffle=False, num_workers=4)
    test_loader  = DataLoader(test_dataset,  batch_size=128, shuffle=False, num_workers=4)

    class_names = full_train.classes
    num_classes = len(class_names)
    use_test = True

# CSV/ZIP-based fallback
else:
    print("Using CSV/ZIP-based dataset...")
    files = entries
    csv_file = next((f for f in files if f.endswith('.csv')), None)
    if csv_file is None:
        zip_file = next((f for f in files if f.endswith('.zip')), None)
        if zip_file is None:
            raise FileNotFoundError("Không tìm thấy CSV hoặc ZIP FER2013 trong BASE_DIR.")
        with zipfile.ZipFile(os.path.join(BASE_DIR, zip_file), 'r') as z:
            z.extractall(BASE_DIR)
        files = os.listdir(BASE_DIR)
        csv_file = next(f for f in files if f.endswith('.csv'))
    csv_path = os.path.join(BASE_DIR, csv_file)
    print("Loading CSV:", csv_path)

    df = pd.read_csv(csv_path)
    pixels = df['pixels'].tolist()
    labels = df['emotion'].tolist()

    X_train, X_val, y_train, y_val = train_test_split(
        pixels, labels, test_size=0.1, stratify=labels, random_state=42
    )
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((48,48)),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    class FER2013Dataset(Dataset):
        def __init__(self, px_list, lbl_list, transform=None):
            self.px = px_list
            self.lbl = lbl_list
            self.transform = transform
        def __len__(self): return len(self.lbl)
        def __getitem__(self, idx):
            arr = np.fromstring(self.px[idx], sep=' ', dtype=np.uint8).reshape(48,48)
            img = Image.fromarray(arr).convert('L')
            if self.transform:
                img = self.transform(img)
            return img, self.lbl[idx]

    train_dataset = FER2013Dataset(X_train, y_train, transform)
    val_dataset   = FER2013Dataset(X_val,   y_val,   transform)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_dataset,   batch_size=128, shuffle=False, num_workers=4)

    class_names = [str(i) for i in sorted(set(labels))]
    num_classes = len(class_names)
    use_test = False

# -- Simple CNN Model Definition --
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# -- Training Loop --
num_epochs = 20
history = {'epoch': [], 'train_loss': [], 'val_loss': [], 'val_acc': []}
for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0
    loop = tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch}/{num_epochs}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        loop.set_postfix(batch_loss=loss.item())
    avg_train_loss = running_loss / len(train_loader.dataset)

    # Validation phase
    model.eval()
    correct = 0
    val_loss_total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs)
            batch_loss = criterion(preds, labels)
            val_loss_total += batch_loss.item() * imgs.size(0)
            correct += (preds.argmax(dim=1) == labels).sum().item()
    avg_val_loss = val_loss_total / len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)
    print(f"Epoch {epoch} -- Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}\n")

    # Store history
    history['epoch'].append(epoch)
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    history['val_acc'].append(val_acc)

# -- Display training history as table --
hist_df = pd.DataFrame(history)
print("\nTraining History:")
print(hist_df.to_string(index=False))

# -- Final Reports & Save --
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        preds = model(imgs).argmax(dim=1).cpu().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels.tolist())
print("\nClassification Report on Validation:")
print(classification_report(all_labels, all_preds, target_names=class_names))

if use_test:
    print("\nEvaluating on Test Set...")
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            preds = model(imgs).argmax(dim=1).cpu().tolist()
            all_preds.extend(preds)
            all_labels.extend(labels.tolist())
    print(f"Test Accuracy: {np.mean(np.array(all_preds) == np.array(all_labels)):.4f}")
    print(classification_report(all_labels, all_preds, target_names=class_names))

out_path = 'simple_cnn_fer2013.pth'
torch.save(model.state_dict(), out_path)
print(f"Model saved to {out_path}")


Entries in BASE_DIR: ['test', 'train']
Using folder-based dataset...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/20: 100%|██████████| 202/202 [02:15<00:00,  1.49it/s, batch_loss=1.49]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1 -- Train Loss: 1.7054, Val Loss: 1.5612, Val Acc: 0.3835



Epoch 2/20:   0%|          | 0/202 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 2/20: 100%|██████████| 202/202 [02:16<00:00,  1.47it/s, batch_loss=1.25]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slownes

Epoch 2 -- Train Loss: 1.4951, Val Loss: 1.4086, Val Acc: 0.4626



...

Epoch 20 -- Train Loss: 0.7828, Val Loss: 1.1801, Val Acc: 0.5949


Training History:
 epoch  train_loss  val_loss  val_acc
     1    1.705364  1.561231 0.383490
     2    1.495112  1.408559 0.462557
     3    1.372897  1.292234 0.493556
     4    1.297161  1.261112 0.516196
     5    1.238031  1.208079 0.540230
     6    1.187049  1.178747 0.547893
     7    1.143619  1.169405 0.550679
     8    1.113902  1.159710 0.567398
     9    1.079663  1.136285 0.575758
    10    1.051811  1.122659 0.579589
    11    1.019979  1.129691 0.571230
    12    0.991499  1.139807 0.570881
    13    0.958768  1.120986 0.582027
    14    0.931808  1.114486 0.585859
    15    0.896291  1.129459 0.586555
    16    0.880783  1.159016 0.571926
    17    0.855153  1.158891 0.586207
    18    0.832244  1.161444 0.577499
    19    0.805876  1.152816 0.587252
    20    0.782822  1.180095 0.594915


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Classification Report on Validation:
              precision    recall  f1-score   support

       angry       0.52      0.49      0.51       376
     disgust       0.74      0.32      0.44        44
        fear       0.47      0.42      0.44       424
       happy       0.81      0.80      0.80       732
     neutral       0.49      0.62      0.55       510
         sad       0.47      0.46      0.46       474
    surprise       0.74      0.73      0.73       311

    accuracy                           0.60      2871
   macro avg       0.61      0.55      0.56      2871
weighted avg       0.60      0.60      0.60      2871


Evaluating on Test Set...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Accuracy: 0.5846
              precision    recall  f1-score   support

       angry       0.53      0.47      0.50       958
     disgust       0.82      0.29      0.43       111
        fear       0.41      0.37      0.39      1024
       happy       0.79      0.79      0.79      1774
     neutral       0.51      0.62      0.56      1233
         sad       0.45      0.44      0.45      1247
    surprise       0.70      0.75      0.72       831

    accuracy                           0.58      7178
   macro avg       0.60      0.53      0.55      7178
weighted avg       0.58      0.58      0.58      7178

Model saved to simple_cnn_fer2013.pth


### Resnet18

In [2]:

# ===== 1. Import Libraries =====
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import zipfile
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report
from tqdm import tqdm
import kagglehub


In [3]:
# ===== 2. Download & Unpack Dataset =====
path = kagglehub.dataset_download("msambare/fer2013")
print("Path to dataset files:", path)
BASE_DIR = path


Path to dataset files: /kaggle/input/fer2013


In [4]:
# ===== 3. Setup Device =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
# ===== 4. Define Transforms =====
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [8]:
# ===== 5. Load Dataset =====
entries = os.listdir(BASE_DIR)
print("Entries in BASE_DIR:", entries)

if 'train' in entries and 'test' in entries:
    # Folder-based
    train_folder = os.path.join(BASE_DIR, 'train')
    test_folder  = os.path.join(BASE_DIR, 'test')
    full_train = ImageFolder(train_folder, transform=transform)
    train_size = int(0.9 * len(full_train))
    val_size   = len(full_train) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_train, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )
    test_dataset = ImageFolder(test_folder, transform=transform)
    use_test = True
else:
    # CSV/ZIP fallback
    csv_file = next((f for f in entries if f.endswith('.csv')), None)
    if not csv_file:
        zip_file = next((f for f in entries if f.endswith('.zip')), None)
        with zipfile.ZipFile(os.path.join(BASE_DIR, zip_file), 'r') as z:
            z.extractall(BASE_DIR)
        entries = os.listdir(BASE_DIR)
        csv_file = next(f for f in entries if f.endswith('.csv'))
    df = pd.read_csv(os.path.join(BASE_DIR, csv_file))
    pixels, labels = df['pixels'].tolist(), df['emotion'].tolist()
    X_train, X_val, y_train, y_val = train_test_split(
        pixels, labels, test_size=0.1, stratify=labels, random_state=42
    )
    class FER2013Dataset(Dataset):
        def __init__(self, px_list, lbl_list, transform=None):
            self.px = px_list
            self.lbl = lbl_list
            self.transform = transform
        def __len__(self): return len(self.lbl)
        def __getitem__(self, idx):
            arr = np.fromstring(self.px[idx], sep=' ', dtype=np.uint8).reshape(48,48)
            img = Image.fromarray(arr).convert('L')
            return self.transform(img), self.lbl[idx]
    train_dataset = FER2013Dataset(X_train, y_train, transform)
    val_dataset   = FER2013Dataset(X_val,   y_val,   transform)
    use_test = False

# Create DataLoaders
batch_size = 128
if use_test:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)
else:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)

class_names = full_train.classes if 'full_train' in locals() else [str(i) for i in sorted(set(labels))]
num_classes = len(class_names)


Entries in BASE_DIR: ['test', 'train']


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:
# ===== 6. Define ResNet-18 Model =====
model = models.resnet18(pretrained=False)
# Adjust input channel & output classes
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc    = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
# ===== 7. Training Loop =====
num_epochs = 20
history = {'epoch': [], 'train_loss': [], 'val_loss': [], 'val_acc': []}
for epoch in range(1, num_epochs+1):
    # Training
    model.train()
    running_loss = 0
    loop = tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch}/{num_epochs}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        loop.set_postfix(batch_loss=loss.item())
    avg_train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_loss += criterion(outputs, labels).item() * imgs.size(0)
            correct  += (outputs.argmax(1) == labels).sum().item()
    avg_val_loss = val_loss / len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {epoch} -- Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
    history['epoch'].append(epoch)
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    history['val_acc'].append(val_acc)


Epoch 1/20: 100%|██████████| 202/202 [08:21<00:00,  2.48s/it, batch_loss=1.47]


Epoch 1 -- Train Loss: 1.5199, Val Loss: 1.3930, Val Acc: 0.4674


Epoch 2/20: 100%|██████████| 202/202 [08:13<00:00,  2.44s/it, batch_loss=1.26]


Epoch 2 -- Train Loss: 1.3089, Val Loss: 1.3134, Val Acc: 0.4963


Epoch 3/20: 100%|██████████| 202/202 [08:24<00:00,  2.50s/it, batch_loss=1.1]


...

Epoch 18 -- Train Loss: 0.2604, Val Loss: 1.7091, Val Acc: 0.5831


Epoch 19/20: 100%|██████████| 202/202 [08:33<00:00,  2.54s/it, batch_loss=0.215]


Epoch 19 -- Train Loss: 0.2345, Val Loss: 1.8088, Val Acc: 0.5646


Epoch 20/20: 100%|██████████| 202/202 [08:28<00:00,  2.52s/it, batch_loss=0.239]


Epoch 20 -- Train Loss: 0.2116, Val Loss: 1.9446, Val Acc: 0.5569


In [12]:
# ===== 8. Evaluation & Save =====
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for imgs, labels in val_loader:
        preds = model(imgs.to(device)).argmax(1).cpu().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels)
print(classification_report(all_labels, all_preds, target_names=class_names))

if 'test_loader' in locals():
    test_preds, test_labels = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            preds = model(imgs.to(device)).argmax(1).cpu().tolist()
            test_preds.extend(preds); test_labels.extend(labels)
    print(f"Test Accuracy: {(np.array(test_preds)==np.array(test_labels)).mean():.4f}")

out_path = 'resnet18_fer2013.pth'
torch.save(model.state_dict(), out_path)
print(f"Model saved to {out_path}")



              precision    recall  f1-score   support

       angry       0.38      0.56      0.45       376
     disgust       0.55      0.36      0.44        44
        fear       0.47      0.48      0.48       424
       happy       0.83      0.70      0.76       732
     neutral       0.47      0.34      0.39       510
         sad       0.41      0.49      0.44       474
    surprise       0.73      0.73      0.73       311

    accuracy                           0.55      2871
   macro avg       0.55      0.52      0.53      2871
weighted avg       0.57      0.55      0.55      2871



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Accuracy: 0.5549
Model saved to resnet18_fer2013.pth
